In [1]:
!pip install transformers datasets peft accelerate scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found 

In [2]:
DATA_PATH = '/kaggle/input/bdm-students'
MODEL_PATH = '/kaggle/input/tinyllama_0510/transformers/default/1'
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [3]:
import pandas as pd
import json

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments,\
    Trainer, DataCollatorForLanguageModeling, pipeline
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import torch

2025-05-11 11:54:06.367123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746964446.551506      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746964446.604130      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
students_df = pd.read_csv(f'{DATA_PATH}/students.csv')
print(students_df.shape)
students_df.head()

(9000, 8)


,name,phone,address,gender,student_id,email,year,department
0,Donald Contreras,9208625450,"478 Mooney Park, New Valerie, VI 28836",male,dc525,dc525@upc.edu,1,POSC
1,Joseph Horton,9886408555,"037 Matthew Shores, Greeneton, CA 98399",male,jh502,jh502@upc.edu,2,RELG
2,Savannah Burns,9047592659,"96124 Lloyd Streets, Edwardmouth, DC 61677",female,sb577,sb577@upc.edu,1,PHAS
3,William Carter,9048473864,"11959 Clark Village, Ivanview, NH 43940",male,wc352,wc352@upc.edu,3,PBSC
4,John Rodriguez,9685225730,"051 Weaver Glen Apt. 724, West Davidborough, M...",male,jr330,jr330@upc.edu,3,SOAN


In [5]:
departments = pd.read_csv(f'{DATA_PATH}/department', sep='\t', encoding_errors='ignore')
print(departments.shape)
departments.head()

(26, 7)


,dept_code,division,name,office_number,building_code,chair_code,blurb
0,ENGL,Arts and Humanities,English,300,WNTP,lstaley,English study at Hudson unlocks rich opportuni...
1,SOAN,Social Sciences,Sociology and Anthropology,390,ALMN,eshever,Join in the ongoing comparative study of human...
2,MATH,Natural Sciences and Mathematics,Mathematics,290,FRLY,dschult,Explore the intrinsic elegance of humanitys tr...
3,ARTS,Arts and Humanities,Art and Art History,387,WHET,emarlowe,"Examine the history, theory, and practice of t..."
4,PHAS,Natural Sciences and Mathematics,Physics and Astronomy,331,FSC,egalvez,The physics and astronomy department explores ...


In [6]:
dept = dict(zip(departments['dept_code'], departments['name']))

In [7]:
n_students = 600
students_group = students_df.sample(n=n_students, replace=True, random_state=42)
students_group['major'] = students_group.apply(lambda x: dept[x['department']], axis=1)
students_group = students_group[['name', 'year', 'major']]
students_group.columns = ['student_name', 'student_year', 'student_major']
print(students_group.shape)
students_group.head()

(600, 3)


,student_name,student_year,student_major
7270,Michael Johnson,2,Art and Art History
860,Alexandria Gonzales,3,Classics
5390,Joseph Mclean,1,Philosophy
5191,Samantha Brown,2,English
5734,Sara Weiss,4,History


In [8]:
students = students_group.to_dict('records')
len(students), students[:2]

(600,
 [{'student_name': 'Michael Johnson',
   'student_year': 2,
   'student_major': 'Art and Art History'},
  {'student_name': 'Alexandria Gonzales',
   'student_year': 3,
   'student_major': 'Classics'}])

In [9]:
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base, MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


In [10]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeo

In [11]:
outputs = []

In [69]:
BATCH_SIZE = 50
i = 11
i*BATCH_SIZE, (i+1)*BATCH_SIZE

(550, 600)

In [70]:
ds = Dataset.from_list(students[i*BATCH_SIZE:(i+1)*BATCH_SIZE])

In [71]:
len(ds), ds[0]

(50,
 {'student_name': 'Eric Hull',
  'student_year': 2,
  'student_major': 'Philosophy'})

In [72]:
def format_prompt(example):
    example_dict = {k: example[k] for k in example.keys()}
    prompt = f"""### Instruction:
{json.dumps(example_dict)}

### Response:"""
    return {"prompt": prompt}

In [73]:
ds = ds.map(format_prompt)
ds[0]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

{'student_name': 'Eric Hull',
 'student_year': 2,
 'student_major': 'Philosophy',
 'prompt': '### Instruction:\n{"student_name": "Eric Hull", "student_year": 2, "student_major": "Philosophy"}\n\n### Response:'}

In [74]:
i

11

In [75]:
output = pipe(ds["prompt"], batch_size=1)
outputs.append(output)

In [76]:
with open(f'/kaggle/working/batch_{i}.txt', 'w') as f:
    for out in output:
        f.write(out[0]['generated_text'] + '\n===\n')